In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
block_size = 8
batch_size = 4
max_iters = 1000
eval_iters = 250
learning_rate = 3e-4

In [2]:
with open('/kaggle/input/wizard-of-oz/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\t', '\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch : i for i, ch in enumerate(chars)}
int_to_string = {i : ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([81,  1,  1,  0,  0,  2,  2, 29, 40, 43, 40, 45, 33, 50,  2, 26, 39, 29,
         2, 45, 33, 30,  2, 48, 34, 51, 26, 43, 29,  2, 34, 39,  2, 40, 51,  1,
         1,  2,  2, 27, 50,  1,  1,  2,  2, 37, 12,  2, 31, 43, 26, 39, 36,  2,
        27, 26, 46, 38,  1,  1,  2,  2, 26, 46, 45, 33, 40, 43,  2, 40, 31,  2,
        45, 33, 30,  2, 48, 34, 51, 26, 43, 29,  2, 40, 31,  2, 40, 51, 10,  2,
        45, 33, 30,  2, 37, 26, 39, 29,  2, 40])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
test_data = data[n:]


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When input is ', context, 'Output is ', target)

When input is  tensor([81]) Output is  tensor(1)
When input is  tensor([81,  1]) Output is  tensor(1)
When input is  tensor([81,  1,  1]) Output is  tensor(0)
When input is  tensor([81,  1,  1,  0]) Output is  tensor(0)
When input is  tensor([81,  1,  1,  0,  0]) Output is  tensor(2)
When input is  tensor([81,  1,  1,  0,  0,  2]) Output is  tensor(2)
When input is  tensor([81,  1,  1,  0,  0,  2,  2]) Output is  tensor(29)
When input is  tensor([81,  1,  1,  0,  0,  2,  2, 29]) Output is  tensor(40)


In [6]:
#Explanation of how batches work
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)
    

inputs:
tensor([[62, 69, 66, 59, 25,  4,  2, 55],
        [79, 69, 75,  2, 74, 77, 69, 10],
        [74,  2, 63, 74,  2, 69, 75, 74],
        [58,  2, 62, 63, 67, 12,  2, 34]])
targets:
tensor([[69, 66, 59, 25,  4,  2, 55, 73],
        [69, 75,  2, 74, 77, 69, 10,  2],
        [ 2, 63, 74,  2, 69, 75, 74, 12],
        [ 2, 62, 63, 67, 12,  2, 34, 74]])


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
#Bigram Language Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self, index, targets = None):
        #logits is like softmax for each index
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            #Unpacks dimensions for batch size (B), sequence length (T), and vocab size (C).
            B, T, C = logits.shape
            #Reshapes logits to combine all words in the batch for efficient loss calculation.
            logits = logits.view(B*T, C)
            #Reshapes targets to match the new logits shape.
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            #Focous only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            #Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(B,T+1)
        return index

model = BigramLanguageModel(vocab_size)
# m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device = device)
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	 h)Pbl,jm853f)S﻿j!U[!Tl2l9E I-E]n8htvOmVIk[59KlM.Ef)qSXRwaiz3u:z3s;!
RGY[dx7lXN_l]&YEsXC1	NqVI:4X;4
c&om[OI4S
&Btr
1:j0Y!nc[tWc!!SU&Ve8ts_.4
cB[O75v"P;4CfYci9PjyZ9E(lE&P6 RwyJX﻿]n	b]3T9e(A.Xy	N&JHM;U[wKdxL88,i0pZ;WKlEu﻿PZwUO&V7M"X:RZB﻿;Om8RGXIyXynWr4YHH!Hf)nYjJqqu﻿iIa03*Xta1,TJw(	2kqN;SL03v.]1CrZyg2sn.*f)6.DU_VOa1]ox6JG?bPJXphBdoc&Q_9E( 4Y6r1o-Rw[31ovuAar6u&*lMqbu&lwZ'r_9X,ka]:(]'8"_YDPWv'S&*do*3x6 I3TRwAiJ])1	aF _Y2t;
LN2UsqK&2W6oK5L7uwuj3:i5Y]'P4cU'3fff8LzYp4geac,6hTcgFRVsMJ)z
12Ut6w
2?Pb
W"p	


In [9]:
# Create a Pytorch optimiser
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, loss: {losses} ')
    #sample a batch of data
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss)
print(loss.item())

step: 0, loss: {'train': tensor(4.8784), 'test': tensor(4.8956)} 
step: 250, loss: {'train': tensor(4.8136), 'test': tensor(4.8439)} 
step: 500, loss: {'train': tensor(4.7404), 'test': tensor(4.7620)} 
step: 750, loss: {'train': tensor(4.7139), 'test': tensor(4.7137)} 
tensor(4.5332, grad_fn=<NllLossBackward0>)
4.533194541931152


In [10]:
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	9* _:dNe:(G2CC1b17RO]nuLVFw CM-y'l?UZ_cku:"cuL
_1,OM0iKCCMBeI3?h"oVx7M,RwyYIVezYYc'-nCysmbGN6gUWX	T,6m﻿cSoeEld
D.GElK(a4bb.mRwXHinhOI	e&aeh!pz_[ps.PqPE]N﻿BEXQEdBzZG*y&v,h-Z4[S(MojSJ]nX2f-,jWfPAFGy)gGck"0 nC!W*7fPvf),hMp_aXSJ!)3a[p;EgYcwf)GV;EzY:Vna7rGnk8]6f)Ki)dw'9&E(&V	1mfdXW6YPRmqHilEvpkL
﻿4hMVZtvbrvwZffFgfx6)Jlw'vmF;4KItKyQ_?VpQonv'B]TK(h_xu2J?8cSiy. ETU VphXuLCApqQag5n5Q0&lGcu&td?Apk!mhnW6ruw((*x rx4KierWK7_v,?1x7 eE(3aEv"yO4[2fy	QHJu﻿ZX-Umnff k:fs[!U﻿(&og[:ALa]	5g2L
'8RCK6hAlE:yf)nqilBo
&OW
